# European call option value
* Sea $X\sim\log\mathcal{N}(\mu,\sigma^2)$ el precio de un portafolio. Si el precio de un contrato de opción compra estilo europeo es $s$ dólares (strike price); entonces el retorno esperado de tomar el contrato es
$$c=\mathrm{E}\{\max(0,X-s)\}$$

## 1. Importar modulos requeridos

In [1]:
import numpy as np # Algebra matricial
from math import sqrt, exp, log, erf, ceil, pi
import matplotlib.pyplot as plt # gráficos
from scipy.stats import norm
from scipy.special import erfinv
import time
import random # Simulacion aleatoria
random.seed( 0 ) # Fijando la semilla (reproducir resultados)
np.random.seed( 0 ) # Fijando la semilla (reproducir resultados)

## 2. Definir algunas functiones auxiliares

### 2.1. Resultado exacto

$$c=\mathrm{E}\{\max(0,X-s)\}=\int_{-\infty}^{\infty}\max(0,\mu-s)\mathrm{d}\mu=\int_{-\infty}^{s}\max(0,\mu-s)\mathrm{d}\mu+\int_{s}^{\infty}\max(0,\mu-s)\mathrm{d}\mu$$
$$c=\int_{s}^{\infty}\mu\mathrm{d}\mu-s\int_{s}^{\infty}\mathrm{d}\mu=\left(\mathrm{E}[X|X\geq s]-s\right)(1-CDF(s))$$

la esperaza condicional y la CDF de la distribución log-normal es conocida
$$\mathrm{E}[X|X\geq s]=\mathrm{e}^{\mu+\frac{\sigma^2}{2}}\frac{\Phi\left(
\frac{\mu+\sigma^2-\log s}{\sigma}
\right)
}{1-\Phi\left(\frac{\log s - \mu}{\sigma}\right)}\text{ y }CDF(s)=\Phi\left(\frac{\log s - \mu}{\sigma}\right)$$


In [2]:
def value_call(mu,sigma2,strike):
    ax1 = norm.cdf( ( mu+sigma2-log(strike) )/sqrt(sigma2) )
    CDF = norm.cdf( ( log(strike)-mu )/sqrt(sigma2) )
    EX_S = exp( mu+sigma2/2 )*( ax1/( 1-CDF ) )
    
    return (EX_S-strike)*(1 - CDF)

### 2.2. Monte Carlo simulation

In [3]:
def mc_call(mu,sigma2,strike,nsample):
    r = np.random.normal(0,1,(nsample,1))
    x = np.exp(sqrt(sigma2)*r+mu)
    c_vec = np.maximum(0,x-strike)
    return np.mean(c_vec)

## 3. Ejercicios

Considere $\mu=5$ y $\sigma^2=0.1$

In [4]:
mu=5
sigma2=0.1

## Strike = 110

* ¿Cuál es el valor de la opción?
* Según la formula presentada en clase, ¿cuánto es un valor razonable para el tamaño de muestra?
* ¿Qué tan buena es la aproximación de Monte Carlo para $n\in\{10, 10^2, 10^3, 10^4, 10^5, 10^6\}?

In [5]:
strike = 110
valor_opcion = value_call(mu,sigma2,strike)
print('Valor real de la opción: %1.3f' %valor_opcion)

Valor real de la opción: 48.813


In [6]:
Alpha = 0.05
Epsilon = 0.01
min_sample = ceil(2*(erfinv(1-Alpha)/Epsilon)**2)
print("Minimum sample size for %1.2f%% confidence level and precision of %1.2fSTDs: %1d" 
      %(100*Alpha,Epsilon,min_sample))

start_time = time.time()
app_opcion = mc_call(mu,sigma2,strike,min_sample)
end_time = time.time()
Error = abs(100*(app_opcion-valor_opcion)/valor_opcion)
print("Approx. con %1d simulaciones: %2.3f, error: %2.5f%%, tiempo de cálculo: %1.5f" 
      %(min_sample,app_opcion,Error,end_time-start_time))

Minimum sample size for 5.00% confidence level and precision of 0.01STDs: 38415
Approx. con 38415 simulaciones: 48.501, error: 0.63828%, tiempo de cálculo: 0.00200


In [7]:
random.seed( 0 ) # Fijando la semilla (reproducir resultados)
np.random.seed( 0 ) # Fijando la semilla (reproducir resultados)
for ii in range(6):
    start_time = time.time()
    app_opcion = mc_call(mu,sigma2,strike,10**(1+ii))
    end_time = time.time()
    Error = abs(100*(app_opcion-valor_opcion)/valor_opcion)
    print("Approx. con 10^%1d simulaciones: %2.3f, error: %2.5f%%, tiempo de cálculo: %1.5f" 
          %(1+ii,app_opcion,Error,end_time-start_time))

Approx. con 10^1 simulaciones: 86.383, error: 76.96917%, tiempo de cálculo: 0.00000
Approx. con 10^2 simulaciones: 50.370, error: 3.19111%, tiempo de cálculo: 0.00000
Approx. con 10^3 simulaciones: 45.751, error: 6.27171%, tiempo de cálculo: 0.00100
Approx. con 10^4 simulaciones: 48.176, error: 1.30350%, tiempo de cálculo: 0.00000
Approx. con 10^5 simulaciones: 48.957, error: 0.29628%, tiempo de cálculo: 0.00585
Approx. con 10^6 simulaciones: 48.850, error: 0.07727%, tiempo de cálculo: 0.08657


## Strike = 300

* ¿Cuál es el valor de la opción?
* Según la formula presentada en clase, ¿cuánto es un valor razonable para el tamaño de muestra?
* ¿Qué tan buena es la aproximación de Monte Carlo para $n\in\{10, 10^2, 10^3, 10^4, 10^5, 10^6\}?

In [8]:
strike = 300
valor_opcion = value_call(mu,sigma2,strike)
print("Valor real de la opción: %1.3f" %valor_opcion)

Valor real de la opción: 0.479


In [9]:
start_time = time.time()
app_opcion = mc_call(mu,sigma2,strike,min_sample)
end_time = time.time()
Error = abs(100*(app_opcion-valor_opcion)/valor_opcion)
print("Approx. con %1d simulaciones: %2.3f, error: %2.5f%%, tiempo de cálculo: %1.5f" 
      %(min_sample,app_opcion,Error,end_time-start_time))

Approx. con 38415 simulaciones: 0.470, error: 1.97120%, tiempo de cálculo: 0.00100


In [10]:
random.seed( 0 ) # Fijando la semilla (reproducir resultados)
np.random.seed( 0 ) # Fijando la semilla (reproducir resultados)
for ii in range(6):
    start_time = time.time()
    app_opcion = mc_call(mu,sigma2,strike,10**(1+ii))
    end_time = time.time()
    Error = abs(100*(app_opcion-valor_opcion)/valor_opcion)
    print("Approx. con 10^%1d simulaciones: %2.3f, error: %2.5f%%, tiempo de cálculo: %1.5f" 
          %(1+ii,app_opcion,Error,end_time-start_time))

Approx. con 10^1 simulaciones: 0.146, error: 69.56093%, tiempo de cálculo: 0.00151
Approx. con 10^2 simulaciones: 0.042, error: 91.18806%, tiempo de cálculo: 0.00000
Approx. con 10^3 simulaciones: 0.357, error: 25.48856%, tiempo de cálculo: 0.00000
Approx. con 10^4 simulaciones: 0.459, error: 4.22243%, tiempo de cálculo: 0.00101
Approx. con 10^5 simulaciones: 0.492, error: 2.61096%, tiempo de cálculo: 0.00300
Approx. con 10^6 simulaciones: 0.481, error: 0.33530%, tiempo de cálculo: 0.04512


## 4. Mejorar la calidad de la simulación MC

### 4.1. Usando antitéticos

Crear la función que transforma la simulación

In [11]:
def normal_antithetic(x,mu): return 2*mu-x

Adaptemos la función ```mc_call()```

In [12]:
def mc_call_anthitetic(mu,sigma2,strike,nsample):
    nsample = ceil(nsample/2)
    r0 = np.random.normal(0,1,(nsample,1))
    r1 = normal_antithetic(r0,0)
    rr = np.r_[r0,r1]
    x = np.exp(sqrt(sigma2)*rr+mu)
    c_vec = np.maximum(0,x-strike)
    return np.mean(c_vec)

Repitamos el ejercicio con $strike=300$

In [13]:
random.seed( 0 ) # Fijando la semilla (reproducir resultados)
np.random.seed( 0 ) # Fijando la semilla (reproducir resultados)
for ii in range(6):
    start_time = time.time()
    app_opcion = mc_call_anthitetic(mu,sigma2,strike,10**(1+ii))
    end_time = time.time()
    Error = abs(100*(app_opcion-valor_opcion)/valor_opcion)
    print("Approx. con 10^%1d simulaciones: %2.3f, error: %2.5f%%, tiempo de cálculo: %1.5f" 
          %(1+ii,app_opcion,Error,end_time-start_time))

Approx. con 10^1 simulaciones: 0.146, error: 69.56093%, tiempo de cálculo: 0.00000
Approx. con 10^2 simulaciones: 0.370, error: 22.88776%, tiempo de cálculo: 0.00000
Approx. con 10^3 simulaciones: 0.350, error: 26.86095%, tiempo de cálculo: 0.00109
Approx. con 10^4 simulaciones: 0.390, error: 18.60848%, tiempo de cálculo: 0.00000
Approx. con 10^5 simulaciones: 0.507, error: 5.86775%, tiempo de cálculo: 0.00458
Approx. con 10^6 simulaciones: 0.475, error: 0.85737%, tiempo de cálculo: 0.05962


### 4.2. Importance sampling

In [14]:
def fx(x): return (1/sqrt(2*pi))*np.exp(-(x**2/2))
def fy(x,u): return (1/sqrt(2*pi))*np.exp(-((x-u)**2/2))
def mc_call_impsam(mu,sigma2,strike,nsample):
    u = log(strike)-sigma2/2
    v = u + np.random.normal(0,1,(nsample,1))
    fx_of_v = fx(v)
    fy_of_v = fy(v,u)
    x_of_v = np.exp(sqrt(sigma2)*v+mu)
    c_vec = np.maximum(0,x_of_v-strike)*fx_of_v/fy_of_v
    return np.mean(c_vec)

In [15]:
random.seed( 0 ) # Fijando la semilla (reproducir resultados)
np.random.seed( 0 ) # Fijando la semilla (reproducir resultados)
for ii in range(6):
    start_time = time.time()
    app_opcion = mc_call_impsam(mu,sigma2,strike,10**(1+ii))
    end_time = time.time()
    Error = abs(100*(app_opcion-valor_opcion)/valor_opcion)
    print("Approx. con 10^%1d simulaciones: %2.3f, error: %2.5f%%, tiempo de cálculo: %1.5f" 
          %(1+ii,app_opcion,Error,end_time-start_time))

Approx. con 10^1 simulaciones: 0.001, error: 99.78355%, tiempo de cálculo: 0.00000
Approx. con 10^2 simulaciones: 0.238, error: 50.25909%, tiempo de cálculo: 0.00132
Approx. con 10^3 simulaciones: 0.492, error: 2.74852%, tiempo de cálculo: 0.00000
Approx. con 10^4 simulaciones: 0.408, error: 14.87529%, tiempo de cálculo: 0.00000
Approx. con 10^5 simulaciones: 0.514, error: 7.22295%, tiempo de cálculo: 0.00601
Approx. con 10^6 simulaciones: 0.480, error: 0.18769%, tiempo de cálculo: 0.12255
